In [1]:
%cd /hdd/yuchen/pipeline/training_pipeline

/hdd/yuchen/pipeline/training_pipeline


In [2]:
from torch_utils.engine import ( train_one_epoch, evaluate, utils)
from torch.utils.data import ( distributed, RandomSampler, SequentialSampler)
from datasets import (
    create_train_dataset, create_valid_dataset, 
    create_train_loader, create_valid_loader)
from models.create_fasterrcnn_model import create_model
from utils.general import (
    set_training_dir, Averager, 
    save_model, save_loss_plot,
    show_tranformed_image,
    save_mAP, save_model_state, SaveBestModel,
    yaml_save, init_seeds)
from utils.logging import (
    set_log, coco_log,set_summary_writer, tensorboard_loss_log,  tensorboard_map_log,
    csv_log, wandb_log,  wandb_save_model, wandb_init)

import torch
import argparse
import yaml
import numpy as np
import torchinfo
import os

In [3]:
parse_opt = {
    'model' : 'fasterrcnn_resnet50_fpn_v2',
    'data' : '',
    'device' : 'cuda:0',
    'epochs' : 5,
    'workers' : 4,
    'batch' : 2,
    'lr' : 0.001,
    'imgsz' : 224,
    'name' : '',
    'vis_transformed' : False,
    'mosaic' : 0.0,
    'use_train_aug' : False,
    'cosine_annealing' : True,  # use cosine annealing warm restarts
    'weights' : None,  # load weights directly from files
    'resume_training' : False,
    'square_training' : True,  # 'Resize images to square shape
    'sync_bn' : True,  # sync batch norm
    'amp' : True,  # use automatic mixed precision
    'seed' : 42,
    'project_dir' : None
}
args = parse_opt

In [4]:
args['epochs'] = 90
args['imgsz'] = 224
args['device'] = 'cuda:1'
args['model'] = 'fasterrcnn_swin'

args['name'] = 'land4sensor_training'
args['batch'] = 8
args['data'] = 'data_configs/land4sensor.yaml'

In [5]:
torch.multiprocessing.set_sharing_strategy('file_system')

RANK = int(os.getenv('RANK', -1))

# For same annotation colors each time.
np.random.seed(42)

with open(args['data']) as file:
    data_configs = yaml.safe_load(file)

init_seeds(args['seed'] + 1 + RANK, deterministic=True)
    
    # Settings/parameters/constants.
TRAIN_DIR_IMAGES = os.path.normpath(data_configs['TRAIN_DIR_IMAGES'])
TRAIN_DIR_LABELS = os.path.normpath(data_configs['TRAIN_DIR_LABELS'])
VALID_DIR_IMAGES = os.path.normpath(data_configs['VALID_DIR_IMAGES'])
VALID_DIR_LABELS = os.path.normpath(data_configs['VALID_DIR_LABELS'])
CLASSES = data_configs['CLASSES']
NUM_CLASSES = data_configs['NC']
NUM_WORKERS = args['workers']
DEVICE = torch.device(args['device'])
print("device",DEVICE)
NUM_EPOCHS = args['epochs']
SAVE_VALID_PREDICTIONS = data_configs['SAVE_VALID_PREDICTION_IMAGES']
BATCH_SIZE = args['batch']
VISUALIZE_TRANSFORMED_IMAGES = args['vis_transformed']
OUT_DIR = set_training_dir(args['name'], args['project_dir'])
COLORS = np.random.uniform(0, 1, size=(len(CLASSES), 3))
SCALER = torch.cuda.amp.GradScaler() if args['amp'] else None

device cuda:1


In [6]:
set_log(OUT_DIR)
writer = set_summary_writer(OUT_DIR)

yaml_save(file_path=os.path.join(OUT_DIR, 'opt.yaml'), data=args)

    # Model configurations
IMAGE_SIZE = args['imgsz']
    
train_dataset = create_train_dataset(
        TRAIN_DIR_IMAGES, 
        TRAIN_DIR_LABELS,
        IMAGE_SIZE, 
        CLASSES,
        use_train_aug=args['use_train_aug'],
        mosaic=args['mosaic'],
        square_training=args['square_training']
    )
valid_dataset = create_valid_dataset(
        VALID_DIR_IMAGES, 
        VALID_DIR_LABELS, 
        IMAGE_SIZE, 
        CLASSES,
        square_training=args['square_training']
    )

train_sampler = RandomSampler(train_dataset)
valid_sampler = SequentialSampler(valid_dataset)

train_loader = create_train_loader(train_dataset, BATCH_SIZE, NUM_WORKERS, batch_sampler=train_sampler)
valid_loader = create_valid_loader(valid_dataset, BATCH_SIZE, NUM_WORKERS)

Checking Labels and images...


  0%|          | 0/1440 [00:00<?, ?it/s]

Checking Labels and images...


  0%|          | 0/309 [00:00<?, ?it/s]

In [7]:
train_loss_hist = Averager()
train_loss_list = []
loss_cls_list = []
loss_box_reg_list = []
loss_objectness_list = []
loss_rpn_list = []
train_loss_list_epoch = []
val_map_05 = []
val_map = []
start_epochs = 0

if args['weights'] is None:
    print('Building model from scratch or load weights from files')
    build_model = create_model[args['model']]
    model = build_model(num_classes=NUM_CLASSES, pretrained=True)

    model.transform.min_size = (args['imgsz'], )
    model = model.to(DEVICE)

    # print(model)

total_params = sum(p.numel() for p in model.parameters())
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=args['lr'], momentum=0.9, nesterov=True)
# optimizer = torch.optim.AdamW(params, lr=0.0001, weight_decay=0.0005)

if args['cosine_annealing']:
    # LR will be zero as we approach `steps` number of epochs each time.
    # If `steps = 5`, LR will slowly reduce to zero every 5 epochs.
    steps = NUM_EPOCHS + 10
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, 
        T_0=steps,
        T_mult=1,
        verbose=False
    )
else:
    scheduler = None

save_best_model = SaveBestModel()

Building model from scratch or load weights from files
current model setting: base_from_mae


/hdd/yuchen/anaconda3/envs/291k/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [ ]:
best_map = -np.inf
best_map_05 = -np.inf
patience = 10
ct = 0


for epoch in range(start_epochs, NUM_EPOCHS):
            
    train_loss_hist.reset()

    _, batch_loss_list, \
            batch_loss_cls_list, \
            batch_loss_box_reg_list, \
            batch_loss_objectness_list, \
            batch_loss_rpn_list = train_one_epoch(
            model, 
            optimizer, 
            train_loader, 
            DEVICE, 
            epoch, 
            train_loss_hist,
            print_freq=100,
                
            scheduler=scheduler,
            scaler=SCALER
        )

    stats, val_pred_image = evaluate(
            model, 
            valid_loader, 
            device=DEVICE,
            save_valid_preds=SAVE_VALID_PREDICTIONS,
            out_dir=OUT_DIR,
            classes=CLASSES,
            colors=COLORS
        )

    # Append the current epoch's batch-wise losses to the `train_loss_list`.
    train_loss_list.extend(batch_loss_list)
    loss_cls_list.append(np.mean(np.array(batch_loss_cls_list,)))
    loss_box_reg_list.append(np.mean(np.array(batch_loss_box_reg_list)))
    loss_objectness_list.append(np.mean(np.array(batch_loss_objectness_list)))
    loss_rpn_list.append(np.mean(np.array(batch_loss_rpn_list)))

    # Append curent epoch's average loss to `train_loss_list_epoch`.
    train_loss_list_epoch.append(train_loss_hist.value)
    val_map_05.append(stats[1])
    val_map.append(stats[0])

    print(stats[0],stats[1] )

    if stats[0] > best_map:
        best_map = stats[0]
        ct = 0
        torch.save({
                'epoch': 1,
                'model_state_dict': model.state_dict(),
                'data':  data_configs,
                'model_name': args['model'],
                }, f"{OUT_DIR}/best_model.pth")
    elif stats[1] > best_map_05:
        best_map_05 = stats[1]
        ct = 0
        torch.save({
                'epoch': 1,
                'model_state_dict': model.state_dict(),
                'data':  data_configs,
                'model_name': args['model'],
                }, f"{OUT_DIR}/best_model.pth")
    else: 
        ct += 1
        if ct > patience: 
            print('End: best val_map and best_map_05', best_map, best_map_05)
            break
            

        # Save loss plot for batch-wise list.
    save_loss_plot(OUT_DIR, train_loss_list)
    save_loss_plot( OUT_DIR,   train_loss_list_epoch,'epochs', 'train loss', save_name='train_loss_epoch')
    save_loss_plot(OUT_DIR, loss_cls_list, 'epochs',  'loss cls',save_name='train_loss_cls')
    save_loss_plot( OUT_DIR, loss_box_reg_list,  'epochs',  'loss bbox reg',save_name='train_loss_bbox_reg')
    save_loss_plot( OUT_DIR, loss_objectness_list, 'epochs', 'loss obj',save_name='train_loss_obj' )
    save_loss_plot( OUT_DIR,  loss_rpn_list, 'epochs','loss rpn bbox', save_name='train_loss_rpn_bbox')
    save_mAP(OUT_DIR, val_map_05, val_map)

    tensorboard_loss_log( 'Train loss',   np.array(train_loss_list_epoch),  writer,  epoch )
    tensorboard_map_log( name='mAP',  val_map_05=np.array(val_map_05), val_map=np.array(val_map), writer=writer,epoch=epoch)

    coco_log(OUT_DIR, stats)
    csv_log(OUT_DIR,  stats,  epoch, train_loss_list,loss_cls_list, loss_box_reg_list, loss_objectness_list, loss_rpn_list )


Epoch: [0]  [  0/180]  eta: 0:12:28  lr: 0.000007  loss: 1.8263 (1.8263)  loss_classifier: 0.7132 (0.7132)  loss_box_reg: 0.0478 (0.0478)  loss_objectness: 0.7206 (0.7206)  loss_rpn_box_reg: 0.3448 (0.3448)  time: 4.1605  data: 0.7174  max mem: 576
Epoch: [0]  [100/180]  eta: 0:00:19  lr: 0.001010  loss: 0.7241 (0.8885)  loss_classifier: 0.1384 (0.1711)  loss_box_reg: 0.1230 (0.1096)  loss_objectness: 0.3089 (0.4352)  loss_rpn_box_reg: 0.1380 (0.1725)  time: 0.2066  data: 0.0034  max mem: 576
Epoch: [0]  [179/180]  eta: 0:00:00  lr: 0.001000  loss: 0.7316 (0.8310)  loss_classifier: 0.1594 (0.1656)  loss_box_reg: 0.1494 (0.1258)  loss_objectness: 0.2876 (0.3780)  loss_rpn_box_reg: 0.1291 (0.1617)  time: 0.2103  data: 0.0038  max mem: 576
Epoch: [0] Total time: 0:00:41 (0.2305 s / it)


creating index...
index created!


Test:  [ 0/39]  eta: 0:00:21  model_time: 0.1959 (0.1959)  evaluator_time: 0.0405 (0.0405)  time: 0.5544  data: 0.3034  max mem: 576
Test:  [38/39]  eta: 0:00:00  model_time: 0.1226 (0.1250)  evaluator_time: 0.0465 (0.0661)  time: 0.1907  data: 0.0032  max mem: 576
Test: Total time: 0:00:08 (0.2071 s / it)


Averaged stats: model_time: 0.1226 (0.1250)  evaluator_time: 0.0465 (0.0661)
Accumulating evaluation results...
DONE (t=0.22s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.010
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.016
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.010
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.006
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.026
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.106
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.096
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium

Epoch: [1]  [  0/180]  eta: 0:01:36  lr: 0.001000  loss: 0.7638 (0.7638)  loss_classifier: 0.1744 (0.1744)  loss_box_reg: 0.1826 (0.1826)  loss_objectness: 0.2946 (0.2946)  loss_rpn_box_reg: 0.1122 (0.1122)  time: 0.5335  data: 0.3066  max mem: 576
Epoch: [1]  [100/180]  eta: 0:00:16  lr: 0.000999  loss: 0.7163 (0.7412)  loss_classifier: 0.1536 (0.1635)  loss_box_reg: 0.1629 (0.1673)  loss_objectness: 0.2620 (0.2691)  loss_rpn_box_reg: 0.1399 (0.1413)  time: 0.2050  data: 0.0035  max mem: 576
Epoch: [1]  [179/180]  eta: 0:00:00  lr: 0.000999  loss: 0.6475 (0.7276)  loss_classifier: 0.1483 (0.1615)  loss_box_reg: 0.1768 (0.1699)  loss_objectness: 0.2286 (0.2590)  loss_rpn_box_reg: 0.1010 (0.1373)  time: 0.2059  data: 0.0035  max mem: 576
Epoch: [1] Total time: 0:00:37 (0.2086 s / it)


...

Epoch: [63]  [  0/180]  eta: 0:01:40  lr: 0.000301  loss: 0.4283 (0.4283)  loss_classifier: 0.0961 (0.0961)  loss_box_reg: 0.1729 (0.1729)  loss_objectness: 0.0793 (0.0793)  loss_rpn_box_reg: 0.0800 (0.0800)  time: 0.5572  data: 0.3295  max mem: 576
Epoch: [63]  [100/180]  eta: 0:00:18  lr: 0.000293  loss: 0.4777 (0.4815)  loss_classifier: 0.1102 (0.1092)  loss_box_reg: 0.1838 (0.1867)  loss_objectness: 0.0986 (0.1053)  loss_rpn_box_reg: 0.0781 (0.0802)  time: 0.2221  data: 0.0037  max mem: 576
Epoch: [63]  [179/180]  eta: 0:00:00  lr: 0.000287  loss: 0.4749 (0.4785)  loss_classifier: 0.1016 (0.1084)  loss_box_reg: 0.1708 (0.1846)  loss_objectness: 0.1018 (0.1042)  loss_rpn_box_reg: 0.0841 (0.0813)  time: 0.2230  data: 0.0037  max mem: 576
Epoch: [63] Total time: 0:00:40 (0.2254 s / it)


creating index...
index created!


Test:  [ 0/39]  eta: 0:00:18  model_time: 0.1334 (0.1334)  evaluator_time: 0.0138 (0.0138)  time: 0.4745  data: 0.3105  max mem: 576
Test:  [38/39]  eta: 0:00:00  model_time: 0.1432 (0.1430)  evaluator_time: 0.0206 (0.0297)  time: 0.1774  data: 0.0033  max mem: 576
Test: Total time: 0:00:07 (0.1888 s / it)


Averaged stats: model_time: 0.1432 (0.1430)  evaluator_time: 0.0206 (0.0297)
Accumulating evaluation results...
DONE (t=0.10s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.361
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.168
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.239
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.065
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.212
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.310
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.287
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium

Epoch: [64]  [  0/180]  eta: 0:01:41  lr: 0.000287  loss: 0.7570 (0.7570)  loss_classifier: 0.1605 (0.1605)  loss_box_reg: 0.3109 (0.3109)  loss_objectness: 0.1533 (0.1533)  loss_rpn_box_reg: 0.1324 (0.1324)  time: 0.5632  data: 0.3294  max mem: 576


In [9]:
!python eval.py --data data_configs/land4sensor.yaml --weights outputs/training/land4sensor_training/best_model.pth --imgsz 224 --device cuda:0  --model fasterrcnn_swin

building models from outputs/training/land4sensor_training/best_model.pth
current model setting: base_from_mae
Checking Labels and images...
100%|█████████████████████████████████████| 309/309 [00:00<00:00, 227415.32it/s]
creating index...
index created!
^C
Traceback (most recent call last):
  File "/hdd/yuchen/pipeline/training_pipeline/eval.py", line 202, in <module>
    stats, _ = evaluate(
  File "/hdd/yuchen/anaconda3/envs/291k/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/hdd/yuchen/pipeline/training_pipeline/torch_utils/engine.py", line 144, in evaluate
    outputs = model(images)
  File "/hdd/yuchen/anaconda3/envs/291k/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/hdd/yuchen/anaconda3/envs/291k/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1520, in _call_impl
    return forward_call

In [10]:
!python inference.py --input /hdd/yuchen/pipeline/training_pipeline/data_land4sensor/valid --data data_configs/land4sensor.yaml --weights outputs/training/land4sensor_training/best_model.pth --imgsz 224 --device cuda:0 --threshold 0.8 --model fasterrcnn_swin --save_img True 

Building from model name arguments...
current model setting: base_from_mae
Test instances: 309
Image 1 done...
--------------------------------------------------
Image 2 done...
--------------------------------------------------
Image 3 done...
--------------------------------------------------
Image 4 done...
--------------------------------------------------
Image 5 done...
--------------------------------------------------
Image 6 done...
--------------------------------------------------
Image 7 done...
--------------------------------------------------
Image 8 done...
--------------------------------------------------
Image 9 done...
--------------------------------------------------
Image 10 done...
--------------------------------------------------
Image 11 done...
--------------------------------------------------
Image 12 done...
--------------------------------------------------
Image 13 done...
--------------------------------------------------
Image 14 done...
-------------